In [1]:
from cnn_globals import *

from cnn_data import cnn_data_split

%matplotlib inline

[INFO    ] Using cmssw CMSSW_10_1_7
[INFO    ] Using numpy 1.14.1
/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs_depscipy/3.0-omkpbe5/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
[INFO    ] Using tensorflow 1.5.0
Using TensorFlow backend.
[INFO    ] Using keras 2.1.4
[INFO    ] .. list devices: [_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456)]
[INFO    ] Using scipy 1.0.0
[INFO    ] Using sklearn 0.19.2


In [2]:
# Import images and labels
#data = cnn_data_split(infile_images, test_size=0.4, shuffle=False, nentries=80000)
data = cnn_data_split(infile_images, test_size=0.4, shuffle=False, nentries=3740000)

(images_px_train, images_px_test, images_ch_train, images_ch_test, labels_train, labels_test, parameters_train, parameters_test) = data

[INFO    ] Loading cnn data from ../test7/histos_tbe.18.npz ...
[INFO    ] Loaded the images with shape (3742978, 50, 2),(3742978, 50, 2)
[INFO    ] Loaded the labels with shape (3742978, 3)
[INFO    ] Loaded the parameters with shape (3742978, 4)
[INFO    ] Loaded # of training and testing events: (2244000, 1496000)


In [3]:
import cnn_models
reload(cnn_models)

<module 'cnn_models' from 'cnn_models.pyc'>

In [4]:
from cnn_models import create_model, save_my_model

model = create_model(lr=learning_rate, clipnorm=gradient_clip_norm, dropout=dropout, use_bn=True, use_dropout=False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 63, 315, 2)   0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 9, 315, 2)    0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 9, 315, 2)    0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 9, 315, 2)    0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_4 (

In [5]:
from cnn_data import parse_image_fn, parse_label_fn
from cnn_data_gen import ImageDataGenerator
from six.moves import range, zip

dummy_images_shape = [images_px_train.shape[0], 1, 1, 1]
dummy_images = np.zeros(dummy_images_shape, dtype=np.float32)
x_train = (dummy_images, [images_px_train, images_ch_train])
y_train = labels_train

image_shape = [n_rows, n_columns, n_channels]
label_shape = [n_classes]

datagen = ImageDataGenerator(featurewise_center=False,
                             samplewise_center=False,
                             featurewise_std_normalization=False,
                             samplewise_std_normalization=False,
                             validation_split=0.1,
                             preprocessing_function_x=parse_image_fn,
                             preprocessing_function_y=parse_label_fn,
                             preprocessing_output_shape_x=image_shape,
                             preprocessing_output_shape_y=label_shape,)

datagen_seed = 2023
datagen_batch_size = 200
#datagen.fit(x_train, seed=datagen_seed)
train_generator = datagen.flow(x_train, y_train, subset='training', batch_size=datagen_batch_size, shuffle=True, seed=datagen_seed)
val_generator = datagen.flow(x_train, y_train, subset='validation', batch_size=datagen_batch_size, shuffle=True, seed=datagen_seed)

In [6]:
assert(K.backend() == 'tensorflow')
assert(K.image_data_format() == 'channels_last')

steps_per_epoch = len(train_generator)
validation_steps = len(val_generator)-1

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, workers=1, validation_data=val_generator, validation_steps=validation_steps)

Epoch 1/5
 2502/10098 [======>.......................] - ETA: 56:46 - loss: 1.9064 - acc: 0.2871 - categorical_accuracy: 0.2871 - top_k_categorical_accuracy: 0.5181

KeyboardInterrupt: 

In [8]:
save_my_model(model, name='model_cnn')